In [9]:
import numpy as np
import h5py
import tifffile
from scipy.ndimage.filters import median_filter
from skimage.filters import threshold_otsu
from collections import Counter
import os

## Processing of a single image

## Loading the HDF5 file and converting to tiff

In [10]:
root_hdf5 = '../../pytorch/fake_images_TI/hdf5'
root_tiff = '../../pytorch/fake_images_TI/tiff'
root_postprocess_tiff = '../../pytorch/fake_images_TI/postprocess_tiff'
files_name = os.listdir(root_hdf5)
print(files_name)


['fake_TI_1001.hdf5', 'fake_TI_10736.hdf5', 'fake_TI_3036.hdf5', 'fake_TI_5071.hdf5', 'fake_TI_7216.hdf5']


In [11]:
for file_name in files_name:
    file_path = os.path.join(root_hdf5, file_name)
    f = h5py.File(file_path,'r')
    my_array = f['data'][()]
    img = my_array[0, 0, :, :, :].astype(np.float32)
    file_name = file_name.split('.')[0]+".tiff"
    # print(name)
    file_path = os.path.join(root_tiff, file_name)
    tifffile.imsave(file_path, img)
    # print(img.shape)

## Denoising and thresholding

In [12]:
files_name = os.listdir(root_tiff)
for file_name in files_name:
    file_path = os.path.join(root_tiff, file_name)
    im_in = tifffile.imread(file_path)

    #apply single pixel denoising
    im_in = median_filter(im_in, size=(3, 3, 3))

    #cutaway outer noise area
    #im_in = im_in[40:240, 40:240, 40:240]

    #Normalize to range zero and one
    im_in = im_in/255.

    #Threshhold Image
    threshold_global_otsu = threshold_otsu(im_in)
    segmented_image = (im_in >= threshold_global_otsu).astype(np.int32)

    #Store as postprocessed image
    file_path = os.path.join(root_postprocess_tiff, file_name.split('.')[0]+'.tiff')

    tifffile.imsave(file_path, segmented_image.astype(np.int32))

## Compute porosity

In [13]:
segmented_image = tifffile.imread("postprocessed_example.tiff")
porc = Counter(segmented_image.flatten())
print(porc)
porosity = porc[0]/float(porc[0]+porc[1])
print("Porosity of the sample: ", porosity)

Counter({1: 6425472, 0: 1574528})
Porosity of the sample:  0.196816
